# 모듈, 라이브러리 불러오기

In [ ]:
!pip install transformers

In [ ]:
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

# 환경세팅

In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# GPU 확인
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

# 데이터 준비

In [ ]:
# 데이터셋 불러오기
import pandas as pd
train = pd.read_excel('/content/drive/MyDrive/데캡디/감성대화/Training/train.xlsx')
sad = pd.read_csv('/content/drive/MyDrive/데캡디/감성대화/Training/슬픈_all.csv')
happy = pd.read_csv('/content/drive/MyDrive/데캡디/감성대화/Training/기쁜_all.csv')
unrest = pd.read_csv('/content/drive/MyDrive/데캡디/감성대화/Training/불안_all.csv')

In [ ]:
# 데이터 label에 맞게 변환
sad['label'] = '슬픔'
happy['label'] = '기쁨'
unrest['label'] = '불안'

# 필요한 column만 추출
sad = sad[ ['label', 'content'] ]
happy = happy[ ['label', 'content'] ]
unrest = unrest[ ['label', 'content'] ]

# 예시 출력
sad

In [ ]:
train.head()

In [ ]:
# 필요한 column만 추출
df = train[ ['감정_대분류', '사람문장1'] ]

# column 이름 변경
df = df.rename(columns={'사람문장1':'content', '감정_대분류':'label'})
df.head()

In [ ]:
# 데이터 합치기
df = pd.concat([df, sad, happy, unrest])

# 필요한 감정만 추출
df = df[(df['label']=="기쁨") | (df['label']=="슬픔") | (df['label']=="불안") | (df['label']=="불안 ") | (df['label']=="기쁨 ")]

In [ ]:
'''
# label encoding
df.loc[(df['label'] == "기쁨"), 'label'] = 0  #발라드 => 0
df.loc[(df['label'] == "불안"), 'label'] = 1  #밤 => 0
df.loc[(df['label'] == "당황"), 'label'] = 2  #슬픈 => 0
df.loc[(df['label'] == "슬픔"), 'label'] = 3  #발라드 => 0
df.loc[(df['label'] == "분노"), 'label'] = 4  #밤 => 0
df.loc[(df['label'] == "상처"), 'label'] = 5  #슬픈 => 0
df.loc[(df['label'] == "불안 "), 'label'] = 1  #슬픈 => 0
df.loc[(df['label'] == "기쁨 "), 'label'] = 0  #슬픈 => 0
'''
# label encoding
df.loc[(df['label'] == "기쁨"), 'label'] = 0  #발라드 => 0
df.loc[(df['label'] == "불안"), 'label'] = 1  #밤 => 0
df.loc[(df['label'] == "슬픔"), 'label'] = 2  #발라드 => 0
df.loc[(df['label'] == "불안 "), 'label'] = 1  #슬픈 => 0
df.loc[(df['label'] == "기쁨 "), 'label'] = 0  #슬픈 => 0



In [ ]:
# label type 변경(숫자로)
df['label'] = pd.to_numeric(df['label'])
df.info()

# BERT 모델 준비

In [ ]:
# bert 학습을 위해 CLS 토근과 SEP 토큰 삽입 (CLS -> classification 태스크, SEP -> 문장 구분)
document_bert = ["[CLS] " + str(s) + " [SEP]" for s in df['content']]
document_bert[:5]


In [ ]:
# BertTokenizer 사용, 모델은 multilingual 모델
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in document_bert]
print(tokenized_texts[0])
'''
from transformers import BertTokenizerFast, EncoderDecoderModel
tokenizer = BertTokenizerFast.from_pretrained("kykim/bertshared-kor-base")
model = EncoderDecoderModel.from_pretrained("kykim/bertshared-kor-base", num_labels=6)
'''

In [ ]:
# MAX_LEN -> padding과 유사한 기법, 문장 길이의 분포를 가지고 많이 나타난 길이를 MAX_LEN으로 설정하면 좋음
MAX_LEN = 32
# token -> ids(숫자)로 변경
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]

In [ ]:
# mask -> softmax 확률 값을 0으로 무기하는 역할, 0이 되면 해당 단어의 정보는 셀프 어텐션에 포함되지 않음
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
print(attention_masks[0])

# 데이터 변환

In [ ]:
# input, label split (8:2)
train_inputs, validation_inputs, train_labels, validation_labels = \
train_test_split(input_ids, df['label'].values, random_state=42, test_size=0.2)

# mask, ids split (8:2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=42, 
                                                       test_size=0.2)

In [ ]:
train_labels

In [ ]:
# 각 값 torch 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [ ]:
# BATCH_SIZE 조절 (16,32,64,128,256,512)
BATCH_SIZE = 32

# Data 불러오는 코드 (dataset, sampler, loader)
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

In [ ]:
# gpu 확인
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

# 최적화 및 하이퍼 파라미터 조정

In [ ]:
# model 불러오기 (multilingual)
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
model.cuda()


In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 20

# 총 훈련 스텝
total_steps = len(train_dataloader) * epochs

# lr 조금씩 감소시키는 스케줄러 (나중에는 cosine 실험해볼 예정)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# 모델 학습

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")

In [ ]:
test = pd.read_excel('/content/drive/MyDrive/데캡디/감성대화/Training/validation.xlsx')